ERROR: Could not find a version that satisfies the requirement en_core_web_trf (from versions: none)
ERROR: No matching distribution found for en_core_web_trf
Note: you may need to restart the kernel to use updated packages.


OSError: [E050] Can't find model 'en_core_web_trf'. It doesn't seem to be a Python package or a valid path to a data directory.

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
df = pd.read_csv("hf://datasets/eric27n/NYT-Connections/Connections_Data.csv")
df = df.dropna()
df.head()

,Game ID,Puzzle Date,Word,Group Name,Group Level,Starting Row,Starting Column
0,1.0,2023-06-12,SNOW,WET WEATHER,0.0,1.0,1.0
1,1.0,2023-06-12,LEVEL,PALINDROMES,3.0,1.0,2.0
2,1.0,2023-06-12,SHIFT,KEYBOARD KEYS,2.0,1.0,3.0
3,1.0,2023-06-12,KAYAK,PALINDROMES,3.0,1.0,4.0
4,1.0,2023-06-12,HEAT,NBA TEAMS,1.0,2.0,1.0


In [15]:
label_encoder = LabelEncoder()
df["Group Name"] = label_encoder.fit_transform(df["Group Name"])


In [16]:
vectorizer = TfidfVectorizer()
X_text = vectorizer.fit_transform(df["Word"])
X_numeric = df[["Game ID", "Group Level", "Starting Row", "Starting Column"]].values

# Combine text and numeric features
X = np.hstack((X_text.toarray(), X_numeric))
y = df["Group Name"]

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

Model Accuracy: 0.02


In [17]:
def predict_category(word, game_id, group_level, row, col):
    word_vector = vectorizer.transform([word]).toarray()
    features = np.hstack((word_vector, np.array([[game_id, group_level, row, col]])))
    predicted_label = model.predict(features)
    return label_encoder.inverse_transform(predicted_label)[0]

# Example usage
print(predict_category("RAIN", 2, 1, 3, 2))

WET WEATHER
